In [96]:
import pickle
import spacy
import re
from itertools import groupby
import string
import tensorflow as tf
import numpy as np
import nltk

In [94]:
model_load = pickle.load(open("/content/drive/MyDrive/FINAL FYP/Islamophobia/model_load.pkl", "rb"))
tokenizer = pickle.load(open("/content/drive/MyDrive/FINAL FYP/Islamophobia/tokenizer.pkl", "rb"))
label_encoder = pickle.load(open("/content/drive/MyDrive/FINAL FYP/Islamophobia/label_encoder.pkl", "rb"))
eng = pickle.load(open("/content/drive/MyDrive/FINAL FYP/Islamophobia/eng.pkl", "rb"))
lemma = pickle.load(open("/content/drive/MyDrive/FINAL FYP/Islamophobia/lemma.pkl", "rb"))
nlp = spacy.load("/content/drive/MyDrive/FINAL FYP/Islamophobia")
stop = pickle.load(open("/content/drive/MyDrive/FINAL FYP/Islamophobia/stop.pkl", "rb"))
rel = pickle.load(open("/content/drive/MyDrive/FINAL FYP/Islamophobia/rel.pkl", "rb"))
terms = pickle.load(open("/content/drive/MyDrive/FINAL FYP/Islamophobia/terms.pkl", "rb"))

In [97]:
def decontracted(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def clean_tweet(tweets):
    preprocess_text = []
    for tweet in tweets:
        tweet = str(tweet).lower()
        tweet = re.sub('https?:\/\/[a-zA-Z0-9@:%._\/+~#=?&;-]*', ' ', tweet)
        tweet = re.sub('\$[a-zA-Z0-9]*', ' ', tweet)
        tweet = re.sub('\@[a-zA-Z0-9]*', ' ', tweet)
        tweet = re.sub('[^a-zA-Z\']', ' ', tweet)
        tweet = re.sub(r'\b(?:\d+|\w)\b\s*', ' ', tweet)
        tweet = decontracted(tweet)
        tweet=[lemma.lemmatize(x) for x in nltk.word_tokenize(tweet) if x not in stop]
        preprocess_text.append(tweet)
    return preprocess_text

In [98]:
list_remove = ['islam','muslim','sex','threesome','exmuslims','fuck','fck','ex-muslim','ex muslim','musalman','ex','jerusalem',
               'iran','makkah','madina','dick','lesbian','nude','nudity','porn','sexism','racism','religion','prayer','pornhub',
               'xxx','xxxxx','xvideos','xnxx','north america','naughty','america','prophet','die','hindu','muslim','hate','kill',
               'masjid','women','narrated','allah','muhammad','say','quran','bible','holy','islamophobia','islamophobic','imam',
               'children','jihaad','jihad','anti-Muslim','anti-Muslim hate','hate crime','social policy','equality','extremism', 
               'muslims in greece','hate muslims','end muslims','sex muslims','fuck muslims','muslims in united states','muslims in usa',
               'muslims in pakistan','muslims in india','india','indian','mosque attacks','attacks','attack','terror','terrorist','terrorism',
               'discrimination','black lifes','ali','namaz','pbuh','worship','god','gods','killer','enemy','palestine','rape']

# clean words in the sentences:
def search(array, element):
    if element in array:
        return True
    else:
        return False
    
def nameMatch(alumniNames, muslimNames):
    matches = False
    name = alumniNames.split(' ')
    lastName=''
    if len(name) == 2:
        lastName = name[1]
        firstName = name[0]
        if search(muslimNames, firstName) or search(muslimNames,lastName):
            matches = True
        else:
            matches=False
    else:
        firstName = name[0]
        if search(muslimNames, firstName):
            matches = True
        else:
            matches=False
    return matches
    
def diff_pos_neg_check(pat,arr):
    result_final=False
    result_binary = False
    result_kmp = False
    string=''
    result = search(arr,pat)

    if result != False:
        result_binary = True
    else:
        pass;
#         for word in arr:
#             result = patternSearching(pat, word)
#             if result == None:
#                 result_kmp = False
#             else:
#                 result_kmp = True
    if result_binary or result_kmp:
        result_final=True
    else:
        result_final=False
    if result_final:
        string='found'
        return string
    else:
        string='not found'
        return string

In [99]:
def test_re(s):
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    return regex.sub(' ', s)
def sim_check(text):
    abusive=False
    religious=False
    ban=False
    clean=False
    check = False
    result_=False
    text_lemma = ''
    result_rel=[]
    result_abusive=[]
    text_clean = []
    
    text = [text]
    for t in text:
        doc = nlp(t)
    for tokens in doc:
        text_lemma = text_lemma+' '+tokens.lemma_
    text_lemma = text_lemma.strip()
    text_lemma = test_re(text_lemma)
    splitted_text = text_lemma.lower().split()
    text_clean.append([word for word in splitted_text if word not in stop])
    text_clean = [x for sublist in text_clean for x in sublist]
    for word in range(len(text_clean)):
        rep_rem = "".join(c for c, _ in groupby(text_clean[word]))
        if eng.check(rep_rem) or (rep_rem in terms) or (rep_rem in rel):
            text_clean[word] = rep_rem
        else:
            pass;
        
    text = " ".join(text_clean)
    for s in text.split():
        try:
            result_abusive.append(diff_pos_neg_check(s, terms))
            result_rel.append(diff_pos_neg_check(s, rel))
        except:
            print('Word length is less')
    
    if ('found' in result_abusive) and ('found' in result_rel):
        ban=True
        abusive=False
        religious=False
    else:
        if 'found' in result_rel:
            religious=True
        else:
            clean=True
        if 'found' in result_abusive:
            abusive=True
        else:
            clean=True

    if ban:
        return 'Strong Islamophobia'
    else:
        if abusive and religious:
            result_=True
        else:
            if religious:
                check=True
            else:
                if abusive:
                    return 'Weak Islamophobia'
                else:
                    return 'No-Islamophobia'    
    if check:
        doc = nlp(text)
        for token in doc:
            if (token.lemma_ in rel and token.dep_ in ('nsubj','dobj')):
                try:
                    if doc[token.i+2].tag_ in ('JJ','VBG','VBZ','VB','VBP','RB','VBD') and doc[token.i+2].text in terms:
                        result_=True
                except:
                    try:
                        if doc[token.i+1].dep_ == 'neg' or doc[token.i+2].dep_ in 'neg':
                            if doc[token.i+1].lemma_ == 'not' or doc[token.i+2].lemma_ == 'not':
                                result_=True
                    except:
                        if doc[token.i-1].tag_ in ('VBG','VBD','VB','VBP','VBZ') and doc[token.i-2].dep_ in ('prep','aux') and (doc[token.i-3].tag_ in ('JJ','RB','NNS') or doc[token.i-2].tag_ in ('JJ','RB','NNS')):
                            result_=True
                        elif doc[token.i-1].tag_ in ('DT') and doc[token.i-1].dep_ in ('dobj','det') and doc[token.i-2].tag_ in ('VBG','VBD'):
                            result_=True
                        elif (doc[token.i-1].tag_ in ('VBG','VBD','VB','VBP','VBZ') or doc[token.i-2].dep_ in ('prep','aux')) and (doc[token.i-3].tag_ in ('JJ','RB') or doc[token.i-2].tag_ in ('JJ','RB')):
                            result_=True
        if result_:
            return 'Strong Islamophobia'
        else:
            return 'No-Islamophobia'
    else:
        pass;

In [100]:
def prep_data(text):
    tokens = tokenizer(text, max_length=150, truncation=True, 
                      padding='max_length', 
                      add_special_tokens=True, 
                      return_tensors='tf')
    tokens = {'input_ids': tf.cast(tokens['input_ids'], tf.float64), 'attention_mask': tf.cast(tokens['attention_mask'], tf.float64)}
    probs = model_load.predict(tokens)[0]
    pred = np.argmax(probs)
    pred = label_encoder.inverse_transform([pred])
    return np.round(probs,3),pred

In [101]:
def self_learning(U,rulesystem):
    model_check_s = False
    model_check_w = False
    model_check_n = False
    check = ''
    rule_answer = ''
    model_answer = ''
    labels = ['Strong Islamophobia', 'Weak Islamophobia', 'No-Islamophobia']
    for i in range(len(U)):
        try:
            sentence = clean_tweet([U[i]])
            for t in sentence:
                text = t
            sent = ' '.join(text)
            o = prep_data(sent)
            rule_answer = rulesystem(sent)
            model_answer = o[1]
            if rule_answer == model_answer:
                return model_answer
            elif rule_answer == labels[0] and model_answer in ('No-Islamophobia','Weak Islamophobia'):
                model_check_s = True
            else:
                if o[0][2]>0.05 and o[0][2]<0.50 and o[0][1] < o[0][2]:
                    model_check_w=True
                elif o[0][2]<0.010 and o[0][1]<0.010:
                    model_check_n=True
        except:
            pass;
        if model_check_w:
            check = 'Weak Islamophobia'
        elif model_check_s:
            check = 'Strong Islamophobia'
        elif model_check_n:
            check = 'No-Islamophobia'
        for label in labels:
            if label in {check,rule_answer,model_answer[0]}:
              return label

In [102]:
self_learning(['i love islam'],rulesystem=sim_check)

'Weak Islamophobia'

In [ ]:
# import pickle
# pickle.dump(stop_words, open('/content/drive/MyDrive/FINAL FYP/Islamophobia/stop_words.pkl','wb'))
# pickle.dump(eng, open('/content/drive/MyDrive/FINAL FYP/Islamophobia/eng.pkl','wb'))
# pickle.dump(lemma, open('/content/drive/MyDrive/FINAL FYP/Islamophobia/lemma.pkl','wb'))
# pickle.dump(nlp, open('/content/drive/MyDrive/FINAL FYP/Islamophobia/nlp.pkl','wb'))
# rel = pd.read_csv('/content/drive/MyDrive/FYP/islamophobia complete/words_rel.csv')
# terms = pd.read_csv('/content/drive/MyDrive/FYP/islamophobia complete/words.csv')
# stop = stopwords.words('english')
# terms=set(terms['0'])
# rel = set(rel['0'])
# terms.remove(np.nan)
# print(rel)
# pickle.dump(stop, open('/content/drive/MyDrive/FINAL FYP/Islamophobia/stop.pkl','wb'))
# pickle.dump(rel, open('/content/drive/MyDrive/FINAL FYP/Islamophobia/rel.pkl','wb'))
# pickle.dump(terms, open('/content/drive/MyDrive/FINAL FYP/Islamophobia/terms.pkl','wb'))
# !apt install -qq enchant
# !pip install pyenchant
# !pip install transformers
# !python -m spacy download en_core_web_lg
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')

In [ ]:
# pickle.dump(decontracted, open('/content/drive/MyDrive/FINAL FYP/Islamophobia/decontracted.pkl','wb'))
# pickle.dump(clean_tweet, open('/content/drive/MyDrive/FINAL FYP/Islamophobia/clean_tweet.pkl','wb'))
# pickle.dump(list_remove, open('/content/drive/MyDrive/FINAL FYP/Islamophobia/list_remove.pkl','wb'))
# pickle.dump(nameMatch, open('/content/drive/MyDrive/FINAL FYP/Islamophobia/nameMatch.pkl','wb'))
# pickle.dump(diff_pos_neg_check, open('/content/drive/MyDrive/FINAL FYP/Islamophobia/diff_pos_neg_check.pkl','wb'))
# pickle.dump(test_re, open('/content/drive/MyDrive/FINAL FYP/Islamophobia/test_re.pkl','wb'))
# pickle.dump(sim_check, open('/content/drive/MyDrive/FINAL FYP/Islamophobia/sim_check.pkl','wb'))
# pickle.dump(prep_data, open('/content/drive/MyDrive/FINAL FYP/Islamophobia/prep_data.pkl','wb'))
# pickle.dump(self_learning, open('/content/drive/MyDrive/FINAL FYP/Islamophobia/self_learning.pkl','wb'))

# decontracted = pickle.load(open("/content/drive/MyDrive/FINAL FYP/Islamophobia/decontracted.pkl", "rb"))
# clean_tweet = pickle.load(open("/content/drive/MyDrive/FINAL FYP/Islamophobia/clean_tweet.pkl", "rb"))
# list_remove = pickle.load(open("/content/drive/MyDrive/FINAL FYP/Islamophobia/list_remove.pkl", "rb"))
# nameMatch = pickle.load(open("/content/drive/MyDrive/FINAL FYP/Islamophobia/nameMatch.pkl", "rb"))
# diff_pos_neg_check = pickle.load(open("/content/drive/MyDrive/FINAL FYP/Islamophobia/diff_pos_neg_check.pkl", "rb"))
# test_re = pickle.load(open("/content/drive/MyDrive/FINAL FYP/Islamophobia/test_re.pkl", "rb"))
# sim_check = pickle.load(open("/content/drive/MyDrive/FINAL FYP/Islamophobia/sim_check.pkl", "rb"))
# prep_data = pickle.load(open("/content/drive/MyDrive/FINAL FYP/Islamophobia/prep_data.pkl", "rb"))
# self_learning = pickle.load(open("/content/drive/MyDrive/FINAL FYP/Islamophobia/self_learning.pkl", "rb"))

In [ ]:
# pickle.dump(model_load,open('/content/drive/MyDrive/FINAL FYP/Islamophobia/model_load.pkl','wb'))
# pickle.dump(tokenizer,open('/content/drive/MyDrive/FINAL FYP/Islamophobia/tokenizer.pkl','wb'))
# pickle.dump(label_encoder,open('/content/drive/MyDrive/FINAL FYP/Islamophobia/label_encoder.pkl','wb'))
# model_load = load_model('/content/drive/MyDrive/MODELS/BERT_ISLAMOPHOBIA/')
# tokenizer = DistilBertTokenizer.from_pretrained('/content/drive/MyDrive/MODELS/BERT_ISLAMOPHOBIA_TOKENIZER/')
# label_encoder = pickle.load(open("/content/drive/MyDrive/MODELS/label_encoder_bert_islamophobia.pkl", "rb"))

# import pandas as pd
# import numpy as np
# import nltk
# import string
# import nltk
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import stopwords
# from sklearn.model_selection  import train_test_split
# from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
# from keras.models import load_model
# import spacy 
# import re
# from itertools import groupby
# from nltk.stem import WordNetLemmatizer
# import enchant
# import pickle
# from transformers import DistilBertTokenizer
# import tensorflow as tf
# eng=enchant.Dict('en_US')
# lemma = WordNetLemmatizer()
# nlp = spacy.load("/content/drive/MyDrive/FINAL FYP/Islamophobia")
